# Capstone Project

# Contents

[1) Introduction](#introduction)  
[2) Data](#data)  
[3) Methodology](#methodology)  
[4.1) Analysis of Delhi](#analyzing-venues-of-delhi)  
[4.2) Analysis of Bangalore](#analyzing-venues-of-bangalore)  
[5) Results and Discussion](#results)  
[6) Conclusion](#conclusion)  

# 1) Introduction <a name="introduction"></a>

This projects aims to compare city of Delhi and Bangalore in India. It compares venues and location data of Delhi and Bangalore.With this we try to find how similar or dissimilar neighbourhoods of these cities are.We consider factors like types of venues like food, venues to visit. This project analyzes the neighborhoods in each of these two cities and tries to understand what is popular in them and what they have to offer to someone who is contemplating to make a choice on seeking a life in either of the metro cities.  

# 2)    Data  <a name="data"></a>

For this study, we needed data about neighborhoods in each of these metro cities. We use data published by the government on postal codes for all India. We download the CSV from following site: https://data.gov.in/resources/all-india-pincode-directory-contact-details-along-latitude-and-longitude.
Here, we download the CSV, filter the csv and convert it into two parts to get csv data of Bangalore and Delhi. After this it is read it into a pandas dataframe.

# 3) Methodology <a name="methodology"></a>

We use the Foursquare API to get the venues of each Neighborhood in Delhi and Bangalore. The Foursquare API allows application developers to interact with the Foursquare platform. We will also use the Foursquare API to find the Top Picks venues in Delhi and Mumbai. This will help us in understanding the nature of life Delhi neighborhoods have to offer.We want to cluster out dataset so we use one hot encoding to convert the categories column of venues to dummy variables so that we can analyse and cluster our data by kmeans clustering algorithm. We then use the extracted dataset of venues for each neigborhood to cluster neighborhoods based on venues types. We use KMeans clustering. We use folium library to display the clusters on map of Delhi and Bangalore. Through this we can analyse how our clusters are divided based on boroughs or division of the city like South Delhi, North Delhi.

# 4.1) Analyzing Venues of Delhi <a name="analyzing-venues-of-delhi"></a>

## i) Importing required packages and loading data

In [7]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#importing folium
!pip install folium
import folium

print('Libraries imported.')

     |████████████████████████████████| 102kB 10.5MB/s ta 0:00:01
Libraries imported.


In [8]:
# The code was removed by Watson Studio for sharing.

,Circle Name,Region Name,Division Name,Office Name,Pincode,OfficeType,Delivery,District,StateName
0,Delhi Circle,NaN,Delhi East Division,Anand Vihar SO,110092,SO,Non Delivery,EAST DELHI,Delhi
1,Delhi Circle,NaN,Delhi East Division,Azad Nagar SO East Delhi,110051,SO,Non Delivery,SHAHDARA,Delhi
2,Delhi Circle,NaN,Delhi East Division,Babarpur SO North East Delhi,110032,SO,Non Delivery,SHAHDARA,Delhi
3,Delhi Circle,NaN,Delhi East Division,Badarpur Khadar BO,110090,BO,Non Delivery,NORTH EAST DELHI,Delhi
4,Delhi Circle,NaN,Delhi East Division,Balbir Nagar SO,110032,SO,Non Delivery,SHAHDARA,Delhi


In [9]:
#Removing columns that are not required
df=df[['Pincode','Office Name','District']]

In [10]:
delhi_df=df.groupby('Pincode')["Office Name"].apply(','.join).reset_index()

In [11]:
delhi_df=pd.merge(delhi_df,df[['Pincode','District']],on='Pincode',how='left')

In [12]:
delhi_df.columns=['Pincode','Neighborhood','District']

In [13]:
delhi_df=delhi_df.drop_duplicates(subset=['Pincode']).reset_index(drop=True)

In [14]:
delhi_df.head()

,Pincode,Neighborhood,District
0,110001,"Baroda House SO,Bengali Market SO,Bhagat Singh...",CENTRAL DELHI
1,110002,"AGCR EXTENSION COUNTER,AGCR SO,Ajmeri Gate Ext...",CENTRAL DELHI
2,110003,"Delhi High Court Extension Counter SO,Delhi Hi...",CENTRAL DELHI
3,110004,Rashtrapati Bhawan SO,CENTRAL DELHI
4,110005,"Anand Parbat Indl Area SO,Anand Parbat SO,Bank...",CENTRAL DELHI


In [15]:
geolocater=Nominatim(user_agent="delhi_agent")
latitude=[]
longitude=[]

for i in range(0,len(delhi_df)):
   # initialize variable to None
   location= None
   

   address =delhi_df['Neighborhood'][i]+", Delhi"
   location = geolocater.geocode(address)
   
   if(location ==None): 
     address =delhi_df['District'][i]
     location = geolocater.geocode(address)
   
   latitude.append(location.latitude)
   longitude.append(location.longitude)

In [16]:
#Adding the columns to the dataframe
delhi_df["Latitude"]=latitude
delhi_df["Longitude"]=longitude

In [17]:
delhi_df.head()

,Pincode,Neighborhood,District,Latitude,Longitude
0,110001,"Baroda House SO,Bengali Market SO,Bhagat Singh...",CENTRAL DELHI,28.698548,77.219391
1,110002,"AGCR EXTENSION COUNTER,AGCR SO,Ajmeri Gate Ext...",CENTRAL DELHI,28.698548,77.219391
2,110003,"Delhi High Court Extension Counter SO,Delhi Hi...",CENTRAL DELHI,28.698548,77.219391
3,110004,Rashtrapati Bhawan SO,CENTRAL DELHI,28.698548,77.219391
4,110005,"Anand Parbat Indl Area SO,Anand Parbat SO,Bank...",CENTRAL DELHI,28.698548,77.219391


# ii) Explore  Neighborhoods¶

In [18]:
import requests

In [19]:
# The code was removed by Watson Studio for sharing.

In [20]:
address ="Delhi"
location = geolocater.geocode(address)

latitude=location.latitude
longitude=location.longitude
print(latitude,",",longitude)

28.6517178 , 77.2219388


In [21]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [22]:
delhi_venues = getNearbyVenues(names=delhi_df['Neighborhood'],
                                   latitudes=delhi_df['Latitude'],
                                   longitudes=delhi_df['Longitude'])

Baroda House SO,Bengali Market SO,Bhagat Singh Market SO,CAT EXTENSION COUNTER,Connaught Place SO,Election Commission SO,Janpath SO,Krishi Bhawan SO,Lady Harding Medical College SO,NGT EXTENSION COUNTER,North Avenue SO,Parliament House SO,Patiala House SO,Pragati Maidan SO,Rail Bhawan SO,Sansad Marg HO,Sansadiya Soudh SO,Secretariat North SO,Shastri Bhawan SO,South Avenue SO,Supreme Court SO,New Delhi GPO,RML EXTN COUNTER NDHO
AGCR EXTENSION COUNTER,AGCR SO,Ajmeri Gate Extn SO,Civic Centre PO,Darya Ganj SO,Gandhi Smarak Nidhi SO,Indraprastha HO,IPEstate SO,Minto Road SO,Rouse Avenue Extension Counter SO
Delhi High Court Extension Counter SO,Delhi High Court SO,Pandara Road SO,Aliganj SO South Delhi,C G O Complex SO,Golf Links SO,Kasturba Nagar SO South Delhi,Lodi Road HO,Pragati Vihar SO,Safdarjung Air Port SO
Rashtrapati Bhawan SO
Anand Parbat Indl Area SO,Anand Parbat SO,Bank Street SO Central Delhi,Desh Bandhu Gupta Road SO,Guru Gobind Singh Marg SO,Karol Bagh SO,Master Prithvi Nath

In [23]:
delhi_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Baroda House SO,Bengali Market SO,Bhagat Singh...",28.698548,77.219391,Tikona Park,28.700102,77.220394,Park
1,"Baroda House SO,Bengali Market SO,Bhagat Singh...",28.698548,77.219391,Kaya Skin Clinic,28.700000,77.216670,Cosmetics Shop
2,"AGCR EXTENSION COUNTER,AGCR SO,Ajmeri Gate Ext...",28.698548,77.219391,Tikona Park,28.700102,77.220394,Park
3,"AGCR EXTENSION COUNTER,AGCR SO,Ajmeri Gate Ext...",28.698548,77.219391,Kaya Skin Clinic,28.700000,77.216670,Cosmetics Shop
4,"Delhi High Court Extension Counter SO,Delhi Hi...",28.698548,77.219391,Tikona Park,28.700102,77.220394,Park


In [24]:
delhi_venues.groupby("Neighborhood").count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"A F Rajokari SO,Rajokari BO",2,2,2,2,2,2
"AGCR EXTENSION COUNTER,AGCR SO,Ajmeri Gate Extn SO,Civic Centre PO,Darya Ganj SO,Gandhi Smarak Nidhi SO,Indraprastha HO,IPEstate SO,Minto Road SO,Rouse Avenue Extension Counter SO",2,2,2,2,2,2
"AKMarket SO,Multani Dhanda SO,Pahar Ganj SO,SRT NAGAR EXTENSION COUNTER,Swami Ram Tirth Nagar SO",2,2,2,2,2,2
"Adrash Nagar SO,Bhalaswa BO,Jahangir Puri A Block SO,Jahangir Puri D Block SO,Jahangir Puri H Block SO,NSMandi SO",2,2,2,2,2,2
"Alipur SO,Bakhtawar Pur BO,Bakoli BO,Hiranki BO,Kadipur BO,Khampur BO,Mukhmelpur BO,Nangal Poona BO,Palla BO",2,2,2,2,2,2
"Anand Parbat Indl Area SO,Anand Parbat SO,Bank Street SO Central Delhi,Desh Bandhu Gupta Road SO,Guru Gobind Singh Marg SO,Karol Bagh SO,Master Prithvi Nath Marg SO,Sat Nagar SO",2,2,2,2,2,2
"Anand Vihar SO,IP Extension SO,Laxmi Nagar Extension SO,Laxmi Nagar SO East Delhi,Mandawali Fazalpur SO,Nirman Vihar SO,Shakarpur SO,Surajmal Vihar SO,Yozna Vihar SO",1,1,1,1,1,1
"Anandvas Shakurpur SO,Maurya Enclave SO,Rani Bagh SO,Saraswati Vihar SO,Shakur Pur I Block SO,Shakurbasti Rs SO,Sri Nagar Colony SO",2,2,2,2,2,2
"Arjun Park PO,Baprola B.O,Daulatpur B.O,Dichaun Kalan B.O,Dindarpur B.O,Durga Park PO,Gopal Nagar SO,Jhatikara B.O,Kair B.O,Khaira B.O,Mitraon B.O,Najafgarh S.O,Pandwala Kalan B.O,Papravat B.O,Rewla Khanpur B.O,SHYAM VIHAR S.O,Surehra B.O,Tilangpur Kotla B.O",2,2,2,2,2,2


## iii) Analyze each Neighborhoods

In [25]:
# one hot encoding
delhi_onehot = pd.get_dummies(delhi_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
delhi_onehot['Neighborhood'] = delhi_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [delhi_onehot.columns[-1]] + list(delhi_onehot.columns[:-1])
delhi_onehot = delhi_onehot[fixed_columns]

delhi_onehot.head()

,Neighborhood,ATM,Bakery,Cosmetics Shop,Garden,Gift Shop,Gym,Light Rail Station,Museum,Park,Platform,Vegetarian / Vegan Restaurant
0,"Baroda House SO,Bengali Market SO,Bhagat Singh...",0,0,0,0,0,0,0,0,1,0,0
1,"Baroda House SO,Bengali Market SO,Bhagat Singh...",0,0,1,0,0,0,0,0,0,0,0
2,"AGCR EXTENSION COUNTER,AGCR SO,Ajmeri Gate Ext...",0,0,0,0,0,0,0,0,1,0,0
3,"AGCR EXTENSION COUNTER,AGCR SO,Ajmeri Gate Ext...",0,0,1,0,0,0,0,0,0,0,0
4,"Delhi High Court Extension Counter SO,Delhi Hi...",0,0,0,0,0,0,0,0,1,0,0


In [26]:
delhi_grouped = delhi_onehot.groupby('Neighborhood').mean().reset_index()
delhi_grouped.head()

,Neighborhood,ATM,Bakery,Cosmetics Shop,Garden,Gift Shop,Gym,Light Rail Station,Museum,Park,Platform,Vegetarian / Vegan Restaurant
0,"A F Rajokari SO,Rajokari BO",0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.5,0.0,0.0,0.0
1,"AGCR EXTENSION COUNTER,AGCR SO,Ajmeri Gate Ext...",0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0
2,"AKMarket SO,Multani Dhanda SO,Pahar Ganj SO,SR...",0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0
3,"Adrash Nagar SO,Bhalaswa BO,Jahangir Puri A Bl...",0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.5,0.0,0.0,0.0
4,"Alipur SO,Bakhtawar Pur BO,Bakoli BO,Hiranki B...",0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.5,0.0,0.0,0.0


# Creating DataFrame of top categories

In [27]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [28]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = delhi_grouped['Neighborhood']

for ind in np.arange(delhi_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(delhi_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"A F Rajokari SO,Rajokari BO",Museum,Garden,Vegetarian / Vegan Restaurant,Platform,Park
1,"AGCR EXTENSION COUNTER,AGCR SO,Ajmeri Gate Ext...",Park,Cosmetics Shop,Vegetarian / Vegan Restaurant,Platform,Museum
2,"AKMarket SO,Multani Dhanda SO,Pahar Ganj SO,SR...",Park,Cosmetics Shop,Vegetarian / Vegan Restaurant,Platform,Museum
3,"Adrash Nagar SO,Bhalaswa BO,Jahangir Puri A Bl...",Museum,Garden,Vegetarian / Vegan Restaurant,Platform,Park
4,"Alipur SO,Bakhtawar Pur BO,Bakoli BO,Hiranki B...",Museum,Garden,Vegetarian / Vegan Restaurant,Platform,Park


# iv) Clustering the neighborhoods

In [29]:
# set number of clusters
kclusters = 5

delhi_grouped_clustering = delhi_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(delhi_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 2, 0, 0, 2, 3, 0, 1, 0], dtype=int32)

In [30]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

delhi_merged = delhi_df

# merge delhi_grouped with delhi_data to add latitude/longitude for each neighborhood
delhi_merged = delhi_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
delhi_merged=delhi_merged[~delhi_merged['Cluster Labels'].isnull()].reset_index(drop=True)

delhi_merged.head() # check the last columns!

,Pincode,Neighborhood,District,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,110001,"Baroda House SO,Bengali Market SO,Bhagat Singh...",CENTRAL DELHI,28.698548,77.219391,2.0,Park,Cosmetics Shop,Vegetarian / Vegan Restaurant,Platform,Museum
1,110002,"AGCR EXTENSION COUNTER,AGCR SO,Ajmeri Gate Ext...",CENTRAL DELHI,28.698548,77.219391,2.0,Park,Cosmetics Shop,Vegetarian / Vegan Restaurant,Platform,Museum
2,110003,"Delhi High Court Extension Counter SO,Delhi Hi...",CENTRAL DELHI,28.698548,77.219391,2.0,Park,Cosmetics Shop,Vegetarian / Vegan Restaurant,Platform,Museum
3,110004,Rashtrapati Bhawan SO,CENTRAL DELHI,28.698548,77.219391,2.0,Park,Cosmetics Shop,Vegetarian / Vegan Restaurant,Platform,Museum
4,110005,"Anand Parbat Indl Area SO,Anand Parbat SO,Bank...",CENTRAL DELHI,28.698548,77.219391,2.0,Park,Cosmetics Shop,Vegetarian / Vegan Restaurant,Platform,Museum


In [31]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(delhi_merged['Latitude'], delhi_merged['Longitude'], delhi_merged['Neighborhood'], delhi_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# v) Examine Clusters

### Cluster 1

In [32]:
delhi_merged.loc[delhi_merged['Cluster Labels'] == 0, delhi_merged.columns[[1] + list(range(5, delhi_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
5,"Delhi GPO,Baratooti SO,Chandni Chowk SO,Chawri...",0.0,Museum,Garden,Vegetarian / Vegan Restaurant,Platform,Park
6,"Delhi University SO,Gulabi Bagh SO,Jawahar Nag...",0.0,Museum,Garden,Vegetarian / Vegan Restaurant,Platform,Park
8,"DrMukerjee Nagar SO,G T B Nagar extension SO,G...",0.0,Museum,Garden,Vegetarian / Vegan Restaurant,Platform,Park
13,"R K Puram Sect1 SO,R K Puram Sect12 SO,R K Pur...",0.0,Museum,Garden,Vegetarian / Vegan Restaurant,Platform,Park
14,"Kidwai Nagar East SO,Kidwai Nagar West SO,Laxm...",0.0,Museum,Garden,Vegetarian / Vegan Restaurant,Platform,Park
19,"Adrash Nagar SO,Bhalaswa BO,Jahangir Puri A Bl...",0.0,Museum,Garden,Vegetarian / Vegan Restaurant,Platform,Park
20,"Anandvas Shakurpur SO,Maurya Enclave SO,Rani B...",0.0,Museum,Garden,Vegetarian / Vegan Restaurant,Platform,Park
21,"C4 Keshav Puram SO North West Delhi,Ganeshpura...",0.0,Museum,Garden,Vegetarian / Vegan Restaurant,Platform,Park
22,"Alipur SO,Bakhtawar Pur BO,Bakoli BO,Hiranki B...",0.0,Museum,Garden,Vegetarian / Vegan Restaurant,Platform,Park
23,"Gurgaon Road SO,IGI Airport SO,Mahipalpur BO,N...",0.0,Museum,Garden,Vegetarian / Vegan Restaurant,Platform,Park


### Cluster 2

In [33]:
delhi_merged.loc[delhi_merged['Cluster Labels'] == 1, delhi_merged.columns[[1] + list(range(5, delhi_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
11,"ESI S.O,Karam Pura S.O,L. M. Nagar Indl. Area ...",1.0,Bakery,ATM,Vegetarian / Vegan Restaurant,Platform,Park
12,"Ashok Nagar S.O (West Delhi),Chand Nagar S.O,C...",1.0,Bakery,ATM,Vegetarian / Vegan Restaurant,Platform,Park
15,"Ashoka Park Extn. S.O,Punjabi Bagh S.O,Punjabi...",1.0,Bakery,ATM,Vegetarian / Vegan Restaurant,Platform,Park
16,"Janta Market S.O,Rajouri Garden J 6 S.O,Rajour...",1.0,Bakery,ATM,Vegetarian / Vegan Restaurant,Platform,Park
27,"Bakkarwala B.O,Hirankudna B.O,Kunwar Singh Nag...",1.0,Bakery,ATM,Vegetarian / Vegan Restaurant,Platform,Park
29,"Arjun Park PO,Baprola B.O,Daulatpur B.O,Dichau...",1.0,Bakery,ATM,Vegetarian / Vegan Restaurant,Platform,Park
37,"D.E.S.U. Colony S.O,Jail Road S.O (West Delhi)...",1.0,Bakery,ATM,Vegetarian / Vegan Restaurant,Platform,Park
38,"D. K. Mohan Garden S.O,Hastal Village B.O,Jeev...",1.0,Bakery,ATM,Vegetarian / Vegan Restaurant,Platform,Park
40,"Jawala Heri B.O,Madipur Slum Quarter S.O,Madip...",1.0,Bakery,ATM,Vegetarian / Vegan Restaurant,Platform,Park
41,"Hari Nagar BE Block S.O,Hari Nagar Dadb Block ...",1.0,Bakery,ATM,Vegetarian / Vegan Restaurant,Platform,Park


### Cluster 3

In [34]:
delhi_merged.loc[delhi_merged['Cluster Labels'] == 2, delhi_merged.columns[[1] + list(range(5, delhi_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Baroda House SO,Bengali Market SO,Bhagat Singh...",2.0,Park,Cosmetics Shop,Vegetarian / Vegan Restaurant,Platform,Museum
1,"AGCR EXTENSION COUNTER,AGCR SO,Ajmeri Gate Ext...",2.0,Park,Cosmetics Shop,Vegetarian / Vegan Restaurant,Platform,Museum
2,"Delhi High Court Extension Counter SO,Delhi Hi...",2.0,Park,Cosmetics Shop,Vegetarian / Vegan Restaurant,Platform,Museum
3,Rashtrapati Bhawan SO,2.0,Park,Cosmetics Shop,Vegetarian / Vegan Restaurant,Platform,Museum
4,"Anand Parbat Indl Area SO,Anand Parbat SO,Bank...",2.0,Park,Cosmetics Shop,Vegetarian / Vegan Restaurant,Platform,Museum
7,"Dada Ghosh Bhawan SO,Patel Nagar East SO,Patel...",2.0,Park,Cosmetics Shop,Vegetarian / Vegan Restaurant,Platform,Museum
9,"Nirman Bhawan SO,Udyog Bhawan SO",2.0,Park,Cosmetics Shop,Vegetarian / Vegan Restaurant,Platform,Museum
10,"IARI SO,Inderpuri SO,National Physical Laborat...",2.0,Park,Cosmetics Shop,Vegetarian / Vegan Restaurant,Platform,Museum
34,"AKMarket SO,Multani Dhanda SO,Pahar Ganj SO,SR...",2.0,Park,Cosmetics Shop,Vegetarian / Vegan Restaurant,Platform,Museum
39,Rajender Nagar SO,2.0,Park,Cosmetics Shop,Vegetarian / Vegan Restaurant,Platform,Museum


### Cluster 4

In [35]:
delhi_merged.loc[delhi_merged['Cluster Labels'] == 3, delhi_merged.columns[[1] + list(range(5, delhi_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
17,"Gandhi Nagar Bazar SO,Gandhi Nagar SO East Del...",3.0,Gym,Vegetarian / Vegan Restaurant,Platform,Park,Museum
54,"Chilla BO,Himmatpuri SO,Kalyanpuri SO,Kalyanva...",3.0,Gym,Vegetarian / Vegan Restaurant,Platform,Park,Museum
55,"Anand Vihar SO,IP Extension SO,Laxmi Nagar Ext...",3.0,Gym,Vegetarian / Vegan Restaurant,Platform,Park,Museum
59,"Ghazipur BO,Mayur Vihar PhIII SO,Vasundhra Enc...",3.0,Gym,Vegetarian / Vegan Restaurant,Platform,Park,Museum


### Cluster 5

In [36]:
delhi_merged.loc[delhi_merged['Cluster Labels'] == 4, delhi_merged.columns[[1] + list(range(5, delhi_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
18,"Babarpur SO North East Delhi,Balbir Nagar SO,B...",4.0,Vegetarian / Vegan Restaurant,Platform,Light Rail Station,Gift Shop,Park
30,"Azad Nagar SO East Delhi,Govind Pura SO,Krishn...",4.0,Vegetarian / Vegan Restaurant,Platform,Light Rail Station,Gift Shop,Park
32,"Bhajan Pura SO,Brahampuri SO,Garhi Mandu BO,Ja...",4.0,Vegetarian / Vegan Restaurant,Platform,Light Rail Station,Gift Shop,Park
58,"Dilshad Garden SO,GTB Hospital SO,Jhilmil HO,J...",4.0,Vegetarian / Vegan Restaurant,Platform,Light Rail Station,Gift Shop,Park


# vi) Checking top venues in Delhi for having fun

In [112]:
address = 'Delhi, India'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

28.6517178 77.2219388


In [64]:
search_query = 'Fun'
print(search_query + ' .... OK!')

Fun .... OK!


In [115]:
# The code was removed by Watson Studio for sharing.

In [67]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ee73e5acb82360791e05573'},
 'response': {'venues': [{'id': '4d6e54b924b56ea84a90fefb',
    'name': 'Fun Cinema',
    'location': {'address': 'moti nagar',
     'lat': 28.656842466137036,
     'lng': 77.13922770766668,
     'labeledLatLngs': [{'label': 'display',
       'lat': 28.656842466137036,
       'lng': 77.13922770766668}],
     'distance': 8099,
     'cc': 'IN',
     'city': 'New Delhi',
     'state': 'Delhi',
     'country': 'India',
     'formattedAddress': ['moti nagar', 'New Delhi', 'Delhi', 'India']},
    'categories': [{'id': '4bf58dd8d48988d180941735',
      'name': 'Multiplex',
      'pluralName': 'Multiplexes',
      'shortName': 'Cineplex',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/movietheater_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1592213279',
    'hasPerk': False},
   {'id': '5015516ce4b06bc0be37c29f',
    'name': 'Fun Cinemas',
    'location': {'addre

In [68]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d180941735', 'name': 'M...",False,4d6e54b924b56ea84a90fefb,moti nagar,IN,New Delhi,India,NaN,8099,"[moti nagar, New Delhi, Delhi, India]","[{'label': 'display', 'lat': 28.65684246613703...",28.656842,77.139228,NaN,Delhi,Fun Cinema,v-1592213279,NaN
1,"[{'id': '4bf58dd8d48988d180941735', 'name': 'M...",False,5015516ce4b06bc0be37c29f,Plot No. 9B 9C,IN,New Delhi,India,"Central Business District, Shahdara",7876,"[Plot No. 9B 9C (Central Business District, Sh...","[{'label': 'display', 'lat': 28.65754482513052...",28.657545,77.302301,110032,Delhi,Fun Cinemas,v-1592213279,NaN
2,"[{'id': '5267e4d9e4b0ec79466e48c8', 'name': 'O...",False,59e5ddcf8c35dc3e5745f263,Dwarka,IN,Delhi,India,Dwarka,6111,"[Dwarka (Dwarka), Delhi 110001, Delhi, India]","[{'label': 'display', 'lat': 28.60328692048269...",28.603287,77.192484,110001,Delhi,Fun,v-1592213279,NaN
3,"[{'id': '4bf58dd8d48988d1e9941735', 'name': 'S...",False,4f8cdabae4b05426eee9d41e,NaN,IN,NaN,India,NaN,6250,[India],"[{'label': 'display', 'lat': 28.62610042123082...",28.626100,77.278865,NaN,NaN,Fun republic,v-1592213279,NaN
4,"[{'id': '4bf58dd8d48988d180941735', 'name': 'M...",False,4b9b4c03f964a520fbfe35e3,V3S Mall,IN,New Delhi,India,Nirman Vihar,6468,"[V3S Mall (Nirman Vihar), New Delhi 110092, De...","[{'label': 'display', 'lat': 28.63696070493917...",28.636961,77.285980,110092,Delhi,Fun Cinemas,v-1592213279,NaN


In [69]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Fun Cinema,Multiplex,moti nagar,IN,New Delhi,India,NaN,8099,"[moti nagar, New Delhi, Delhi, India]","[{'label': 'display', 'lat': 28.65684246613703...",28.656842,77.139228,NaN,Delhi,4d6e54b924b56ea84a90fefb
1,Fun Cinemas,Multiplex,Plot No. 9B 9C,IN,New Delhi,India,"Central Business District, Shahdara",7876,"[Plot No. 9B 9C (Central Business District, Sh...","[{'label': 'display', 'lat': 28.65754482513052...",28.657545,77.302301,110032,Delhi,5015516ce4b06bc0be37c29f
2,Fun,Other Event,Dwarka,IN,Delhi,India,Dwarka,6111,"[Dwarka (Dwarka), Delhi 110001, Delhi, India]","[{'label': 'display', 'lat': 28.60328692048269...",28.603287,77.192484,110001,Delhi,59e5ddcf8c35dc3e5745f263
3,Fun republic,Ski Area,NaN,IN,NaN,India,NaN,6250,[India],"[{'label': 'display', 'lat': 28.62610042123082...",28.626100,77.278865,NaN,NaN,4f8cdabae4b05426eee9d41e
4,Fun Cinemas,Multiplex,V3S Mall,IN,New Delhi,India,Nirman Vihar,6468,"[V3S Mall (Nirman Vihar), New Delhi 110092, De...","[{'label': 'display', 'lat': 28.63696070493917...",28.636961,77.285980,110092,Delhi,4b9b4c03f964a520fbfe35e3
5,Fun Cinemas,Multiplex,Pitampura,IN,Delhi,India,Netaji Subhash Place,8428,"[Pitampura (Netaji Subhash Place), Delhi 11003...","[{'label': 'display', 'lat': 28.69185181838334...",28.691852,77.148766,110034,Delhi,4d0e01163905b1f70b859d4b
6,Fun Cinemas CRM,Multiplex,Cross River Mall,IN,NaN,India,NaN,7908,"[Cross River Mall, India]","[{'label': 'display', 'lat': 28.66018802591894...",28.660188,77.302318,NaN,NaN,4ebf915a8231b62f42457992
7,Fun Cinema,None,"Plot No.F-1, 2-3, District Centre, Netaji Subh...",IN,Delhi,India,NaN,8788,"[Plot No.F-1, 2-3, District Centre, Netaji Sub...","[{'label': 'display', 'lat': 28.68751056511828...",28.687511,77.141733,NaN,Delhi,4d3d445b14aa8cfaac83b15e
8,Aggarwal Funcity Mall,Shopping Mall,NaN,IN,New Delhi,India,CBD,7519,"[CBD, New Delhi, Delhi, India]","[{'label': 'display', 'lat': 28.66063529205805...",28.660635,77.298245,NaN,Delhi,4df4ed33d4c01ff6b2f13787
9,Fun City Mall,Shopping Mall,Prashant Vihar,IN,New Delhi,India,NaN,10778,"[Prashant Vihar, New Delhi, Delhi, India]","[{'label': 'display', 'lat': 28.71267298542188...",28.712673,77.136188,NaN,Delhi,4e54c4bcd4c0fe03427d8fe5


In [70]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Delhi

# add the fun venues as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

In [72]:
delhi_fun=dataframe_filtered['categories'].value_counts()
delhi_fun.index.name='categories'
delhi_fun.to_frame().rename(columns={'categories':"Count"},inplace=True)
delhi_fun.head(10)

categories
Multiplex           9
Office              4
Water Park          3
Indoor Play Area    3
Arcade              2
Clothing Store      2
Movie Theater       2
Shopping Mall       2
Wedding Hall        1
Medical Center      1
Name: categories, dtype: int64

# 4.2) Analyzing Banglore Data <a name="analyzing-venues-of-bangalore"></a>

In [73]:
# @hideen cell 
body = client_d09a298a6c884b7283b99fa685aca3ac.get_object(Bucket='gettingstartedwithcapstoneproject-donotdelete-pr-pzx70cudqbmokf',Key='Banglore csv.xlsx')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df2 = pd.read_excel(body)
df2.head()

,Circle Name,Region Name,Division Name,Office Name,Pincode,OfficeType,Delivery,District,StateName
0,Karnataka Circle,NaN,BANGALORE GPO DIVISION,Bengaluru G.P.O.,560001,HO,Delivery,BENGALURU,Karnataka
1,Karnataka Circle,NaN,Bengaluru East Division,A F Station Yelahanka S.O,560063,SO,Delivery,BENGALURU,Karnataka
2,Karnataka Circle,NaN,Bengaluru East Division,Air Force Hospital S.O,560007,SO,Non Delivery,BENGALURU,Karnataka
3,Karnataka Circle,NaN,Bengaluru East Division,Amruthahalli B.O,560092,BO,Non Delivery,BENGALURU,Karnataka
4,Karnataka Circle,NaN,Bengaluru East Division,Anandanagar S.O,560024,SO,Non Delivery,BENGALURU,Karnataka


In [74]:
#Removing columns that are not required
df2=df2[['Pincode','Office Name','District']]

In [75]:
bangalore_df=df2.groupby('Pincode')["Office Name"].apply(','.join).reset_index()

In [76]:
bangalore_df=pd.merge(bangalore_df,df2[['Pincode','District']],on='Pincode',how='left')

In [77]:
bangalore_df.columns=['Pincode','Neighborhood','District']

In [78]:
bangalore_df=bangalore_df.drop_duplicates(subset=['Pincode']).reset_index(drop=True)

In [79]:
bangalore_df.head()

,Pincode,Neighborhood,District
0,560001,"Bengaluru G.P.O.,CMM Court Complex S.O,Dr. Amb...",BENGALURU
1,560002,"Bengaluru Corporation Building S.O,Bengaluru C...",BENGALURU
2,560003,"Malleswaram S.O,Palace Guttahalli S.O,Swimming...",BENGALURU
3,560004,"Basavanagudi H.O,Gaviopuram Extension S.O,Mava...",BENGALURU
4,560005,Fraser Town S.O,BENGALURU


In [80]:
geolocater=Nominatim(user_agent="bangalore_agent")
latitude=[]
longitude=[]

for i in range(0,len(bangalore_df)):
   # initialize variable to None
   location= None
   

   address =bangalore_df['Neighborhood'][i]+", Bangalore"
   location = geolocater.geocode(address)
   
   if(location ==None): 
     address =bangalore_df['District'][i]
     location = geolocater.geocode(address)
   
   latitude.append(location.latitude)
   longitude.append(location.longitude)

In [81]:
#Adding the columns to the dataframe
bangalore_df["Latitude"]=latitude
bangalore_df["Longitude"]=longitude

bangalore_df.head()

,Pincode,Neighborhood,District,Latitude,Longitude
0,560001,"Bengaluru G.P.O.,CMM Court Complex S.O,Dr. Amb...",BENGALURU,12.97912,77.5913
1,560002,"Bengaluru Corporation Building S.O,Bengaluru C...",BENGALURU,12.97912,77.5913
2,560003,"Malleswaram S.O,Palace Guttahalli S.O,Swimming...",BENGALURU,12.97912,77.5913
3,560004,"Basavanagudi H.O,Gaviopuram Extension S.O,Mava...",BENGALURU,12.97912,77.5913
4,560005,Fraser Town S.O,BENGALURU,12.97912,77.5913


# ii) Explore Neighborhoods

In [82]:
# The code was removed by Watson Studio for sharing.

In [83]:
address ="Bangalore"
location = geolocater.geocode(address)

latitude=location.latitude
longitude=location.longitude
print(latitude,",",longitude)

12.9791198 , 77.5912997


In [84]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [85]:
bangalore_venues = getNearbyVenues(names=bangalore_df['Neighborhood'],
                                   latitudes=bangalore_df['Latitude'],
                                   longitudes=bangalore_df['Longitude'])


Bengaluru G.P.O.,CMM Court Complex S.O,Dr. Ambedkar Veedhi S.O,HighCourt S.O,Legislators Home S.O,Mahatma Gandhi Road S.O,Rajbhavan S.O,Vasanthanagar S.O,Vidhana Soudha S.O
Bengaluru Corporation Building S.O,Bengaluru City S.O
Malleswaram S.O,Palace Guttahalli S.O,Swimming Pool Extn S.O,Vyalikaval Extn S.O
Basavanagudi H.O,Gaviopuram Extension S.O,Mavalli S.O,Pampamahakavi Road S.O
Fraser Town S.O
J C Nagar S.O,Training Command IAF S.O
Air Force Hospital S.O
H A L II Stage H.O,Halasur Bazar S.O
Bengaluru Dist Offices Bldg S.O,K. G. Road S.O
Industrial Estate S.O (Bengaluru),Rajajinagar H.O,Rajajinagar IVth Block S.O
Jayangar III Block S.O,Madhavan Park S.O
Science Institute S.O
Jalahalli East S.O,Jalahalli H.O
Jalahalli West S.O
Doorvaninagar S.O,Krishnarajapuram R S S.O,Ramamurthy Nagar S.O
N A L S.O,Vimanapura S.O
Chamrajpet S.O (Bengaluru),Goripalya SO
Seshadripuram S.O
Gayathrinagar S.O,Srirampuram S.O
Yeshwanthpur Bazar S.O,Yeswanthpura S.O
Magadi Road S.O
Anandanagar S.O,H A Farm

In [86]:
bangalore_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Bengaluru G.P.O.,CMM Court Complex S.O,Dr. Amb...",12.97912,77.5913,Vidhana Soudha,12.979027,77.591881,Capitol Building
1,"Bengaluru G.P.O.,CMM Court Complex S.O,Dr. Amb...",12.97912,77.5913,Cubbon Park,12.977042,77.595277,Park
2,"Bengaluru G.P.O.,CMM Court Complex S.O,Dr. Amb...",12.97912,77.5913,Vidhana Soudha Metro,12.978927,77.591851,Metro Station
3,"Bengaluru G.P.O.,CMM Court Complex S.O,Dr. Amb...",12.97912,77.5913,24 Carats,12.981980,77.594564,Hotel
4,"Bengaluru Corporation Building S.O,Bengaluru C...",12.97912,77.5913,Vidhana Soudha,12.979027,77.591881,Capitol Building


In [87]:
bangalore_venues.groupby("Neighborhood").count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"A F Station Yelahanka S.O,BSF Campus Yelahanka S.O",4,4,4,4,4,4
Adugodi S.O,5,5,5,5,5,5
"Agara B.O,Koramangala I Block S.O,Koramangala S.O,St. Johns Medical College S.O",4,4,4,4,4,4
Air Force Hospital S.O,4,4,4,4,4,4
"Amruthahalli B.O,Byatarayanapura B.O,Kodigehalli B.O,Sahakaranagar S.O",4,4,4,4,4,4
"Anandanagar S.O,H A Farm S.O,Hebbal Kempapura S.O",4,4,4,4,4,4
"Anekal S.O,Bestamaranahalli B.O,Byagadadenahalli B.O,Guddanahalli B.O,Indalawadi B.O,Marasur B.O,Samandur B.O,Sidihoskote B.O,Thammanayakanahalli B.O,Vanakanahalli B.O",4,4,4,4,4,4
Anjanapura S.O,1,1,1,1,1,1
Arabic College S.O,4,4,4,4,4,4


# iii) Analyze each Neighborhoods

In [88]:
# one hot encoding
bangalore_onehot = pd.get_dummies(bangalore_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
bangalore_onehot['Neighborhood'] = bangalore_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [bangalore_onehot.columns[-1]] + list(bangalore_onehot.columns[:-1])
bangalore_onehot = bangalore_onehot[fixed_columns]

bangalore_onehot.head()

,Neighborhood,ATM,Airport,Airport Service,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bar,Basketball Court,Beer Bar,Brewery,Burger Joint,Bus Station,Café,Capitol Building,Chaat Place,Chettinad Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Deli / Bodega,Department Store,Dessert Shop,Dhaba,Doner Restaurant,Donut Shop,Duty-free Shop,Farmers Market,Fast Food Restaurant,Flower Shop,Food & Drink Shop,Food Court,French Restaurant,Fruit & Vegetable Store,General Entertainment,Hotel,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Juice Bar,Karnataka Restaurant,Kids Store,Liquor Store,Lounge,Mediterranean Restaurant,Metro Station,Miscellaneous Shop,Movie Theater,Multiplex,North Indian Restaurant,Park,Pizza Place,Playground,Pool,Pub,Punjabi Restaurant,Rajasthani Restaurant,Restaurant,Sandwich Place,Seafood Restaurant,Shopping Mall,Snack Place,South Indian Restaurant,Sports Bar,Steakhouse,Taxi Stand,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,"Bengaluru G.P.O.,CMM Court Complex S.O,Dr. Amb...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Bengaluru G.P.O.,CMM Court Complex S.O,Dr. Amb...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Bengaluru G.P.O.,CMM Court Complex S.O,Dr. Amb...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Bengaluru G.P.O.,CMM Court Complex S.O,Dr. Amb...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Bengaluru Corporation Building S.O,Bengaluru C...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [89]:
bangalore_grouped = bangalore_onehot.groupby('Neighborhood').mean().reset_index()
bangalore_grouped.head()

,Neighborhood,ATM,Airport,Airport Service,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bar,Basketball Court,Beer Bar,Brewery,Burger Joint,Bus Station,Café,Capitol Building,Chaat Place,Chettinad Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Deli / Bodega,Department Store,Dessert Shop,Dhaba,Doner Restaurant,Donut Shop,Duty-free Shop,Farmers Market,Fast Food Restaurant,Flower Shop,Food & Drink Shop,Food Court,French Restaurant,Fruit & Vegetable Store,General Entertainment,Hotel,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Juice Bar,Karnataka Restaurant,Kids Store,Liquor Store,Lounge,Mediterranean Restaurant,Metro Station,Miscellaneous Shop,Movie Theater,Multiplex,North Indian Restaurant,Park,Pizza Place,Playground,Pool,Pub,Punjabi Restaurant,Rajasthani Restaurant,Restaurant,Sandwich Place,Seafood Restaurant,Shopping Mall,Snack Place,South Indian Restaurant,Sports Bar,Steakhouse,Taxi Stand,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,"A F Station Yelahanka S.O,BSF Campus Yelahanka...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Adugodi S.O,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.2,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Agara B.O,Koramangala I Block S.O,Koramangala ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Air Force Hospital S.O,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Amruthahalli B.O,Byatarayanapura B.O,Kodigehal...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Creating DataFrame of top categories

In [90]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


In [91]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = bangalore_grouped['Neighborhood']

for ind in np.arange(delhi_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(bangalore_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"A F Station Yelahanka S.O,BSF Campus Yelahanka...",Park,Hotel,Capitol Building,Metro Station,Dhaba
1,Adugodi S.O,Ice Cream Shop,Playground,Athletics & Sports,Café,Bus Station
2,"Agara B.O,Koramangala I Block S.O,Koramangala ...",Park,Hotel,Capitol Building,Metro Station,Dhaba
3,Air Force Hospital S.O,Park,Hotel,Capitol Building,Metro Station,Dhaba
4,"Amruthahalli B.O,Byatarayanapura B.O,Kodigehal...",Park,Hotel,Capitol Building,Metro Station,Dhaba


# iv) Clustering the neighborhoods

In [92]:
# set number of clusters
kclusters = 5

bangalore_grouped_clustering = bangalore_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bangalore_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]


array([0, 1, 0, 0, 0, 0, 0, 3, 0, 0], dtype=int32)

In [93]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

bangalore_merged = bangalore_df

# merge bangalore_grouped with bangalore_data to add latitude/longitude for each neighborhood
bangalore_merged = bangalore_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
bangalore_merged=bangalore_merged[~bangalore_merged['Cluster Labels'].isnull()].reset_index(drop=True)
bangalore_merged=bangalore_merged[~bangalore_merged['1st Most Common Venue'].isnull()].reset_index(drop=True)

bangalore_merged.head() # check the last columns!


,Pincode,Neighborhood,District,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,560001,"Bengaluru G.P.O.,CMM Court Complex S.O,Dr. Amb...",BENGALURU,12.97912,77.5913,0.0,Park,Hotel,Capitol Building,Metro Station,Dhaba
1,560002,"Bengaluru Corporation Building S.O,Bengaluru C...",BENGALURU,12.97912,77.5913,0.0,Park,Hotel,Capitol Building,Metro Station,Dhaba
2,560004,"Basavanagudi H.O,Gaviopuram Extension S.O,Mava...",BENGALURU,12.97912,77.5913,0.0,Park,Hotel,Capitol Building,Metro Station,Dhaba
3,560005,Fraser Town S.O,BENGALURU,12.97912,77.5913,0.0,Park,Hotel,Capitol Building,Metro Station,Dhaba
4,560007,Air Force Hospital S.O,BENGALURU,12.97912,77.5913,0.0,Park,Hotel,Capitol Building,Metro Station,Dhaba


In [94]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bangalore_merged['Latitude'], bangalore_merged['Longitude'], bangalore_merged['Neighborhood'], bangalore_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# v) Examine Clusters

### Cluster 1

In [95]:
bangalore_merged.loc[bangalore_merged['Cluster Labels'] == 0, bangalore_merged.columns[[1] + list(range(5, bangalore_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Bengaluru G.P.O.,CMM Court Complex S.O,Dr. Amb...",0.0,Park,Hotel,Capitol Building,Metro Station,Dhaba
1,"Bengaluru Corporation Building S.O,Bengaluru C...",0.0,Park,Hotel,Capitol Building,Metro Station,Dhaba
2,"Basavanagudi H.O,Gaviopuram Extension S.O,Mava...",0.0,Park,Hotel,Capitol Building,Metro Station,Dhaba
3,Fraser Town S.O,0.0,Park,Hotel,Capitol Building,Metro Station,Dhaba
4,Air Force Hospital S.O,0.0,Park,Hotel,Capitol Building,Metro Station,Dhaba
5,"Bengaluru Dist Offices Bldg S.O,K. G. Road S.O",0.0,Park,Hotel,Capitol Building,Metro Station,Dhaba
6,"Doorvaninagar S.O,Krishnarajapuram R S S.O,Ram...",0.0,Park,Hotel,Capitol Building,Metro Station,Dhaba
7,"Chamrajpet S.O (Bengaluru),Goripalya SO",0.0,Park,Hotel,Capitol Building,Metro Station,Dhaba
8,"Anandanagar S.O,H A Farm S.O,Hebbal Kempapura S.O",0.0,Park,Hotel,Capitol Building,Metro Station,Dhaba
9,"CMP Centre And School S.O,Museum Road S.O,Rich...",0.0,Park,Hotel,Capitol Building,Metro Station,Dhaba


### Cluster 2

In [96]:
bangalore_merged.loc[bangalore_merged['Cluster Labels'] == 1, bangalore_merged.columns[[1] + list(range(5, bangalore_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
12,Adugodi S.O,1.0,Ice Cream Shop,Playground,Athletics & Sports,Café,Bus Station
20,Benson Town S.O,1.0,Café,Asian Restaurant,Intersection,Bakery,Vietnamese Restaurant
35,Domlur S.O,1.0,Indian Restaurant,Café,Vietnamese Restaurant,Restaurant,Chinese Restaurant
44,C V Raman Nagar S.O,1.0,Indian Restaurant,Pizza Place,Park,Café,Department Store
49,Begur,1.0,Fruit & Vegetable Store,Playground,Bus Station,Dhaba,Clothing Store
52,Bengaluru International Airport S.O,1.0,Airport Service,Coffee Shop,Café,Beer Bar,French Restaurant


### Cluster 3

In [97]:
bangalore_merged.loc[bangalore_merged['Cluster Labels'] == 2, bangalore_merged.columns[[1] + list(range(5, bangalore_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
60,Dasanapura B.O,2.0,North Indian Restaurant,Fruit & Vegetable Store,Chinese Restaurant,Clothing Store,Coffee Shop


### Cluster 4

In [98]:
bangalore_merged.loc[bangalore_merged['Cluster Labels'] == 3, bangalore_merged.columns[[1] + list(range(5, bangalore_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
48,Anjanapura S.O,3.0,Pool,Vietnamese Restaurant,Chettinad Restaurant,Clothing Store,Coffee Shop


### Cluster 5

In [99]:
bangalore_merged.loc[bangalore_merged['Cluster Labels'] == 4, bangalore_merged.columns[[1] + list(range(5, bangalore_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
40,Chandapura S.O,4.0,Asian Restaurant,Department Store,Vietnamese Restaurant,Doner Restaurant,Clothing Store
47,Bellandur S.O,4.0,Department Store,Kids Store,Flower Shop,Asian Restaurant,Vietnamese Restaurant


# vi) Checking top venues for having fun in Bangalore

In [116]:
address = 'Bangalore, India'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

12.9791198 77.5912997


In [101]:
search_query = 'Fun'
print(search_query + ' .... OK!')

Fun .... OK!


In [102]:
# The code was removed by Watson Studio for sharing.

In [117]:
# @hidden cell
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, LIMIT)

In [104]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ee7434dcb82360791ec6681'},
 'response': {'venues': [{'id': '4b755b26f964a52005082ee3',
    'name': 'Fun Cinemas',
    'location': {'address': 'Sigma Mall',
     'crossStreet': 'Cunningham Rd.',
     'lat': 12.988325744377738,
     'lng': 77.59476868660298,
     'labeledLatLngs': [{'label': 'display',
       'lat': 12.988325744377738,
       'lng': 77.59476868660298}],
     'distance': 1091,
     'cc': 'IN',
     'city': 'Bangalore',
     'state': 'Karnātaka',
     'country': 'India',
     'formattedAddress': ['Sigma Mall (Cunningham Rd.)',
      'Bangalore',
      'Karnātaka',
      'India']},
    'categories': [{'id': '4bf58dd8d48988d180941735',
      'name': 'Multiplex',
      'pluralName': 'Multiplexes',
      'shortName': 'Cineplex',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/movietheater_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1592214545',
    'hasPerk': False},
   {'i

In [105]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d180941735', 'name': 'M...",False,4b755b26f964a52005082ee3,Sigma Mall,IN,Bangalore,India,Cunningham Rd.,1091,"[Sigma Mall (Cunningham Rd.), Bangalore, Karnā...","[{'label': 'display', 'lat': 12.98832574437773...",12.988326,77.594769,NaN,Karnātaka,Fun Cinemas,v-1592214545
1,"[{'id': '4f04b25d2fb6e1c99f3db0c0', 'name': 'T...",False,515d73cff136b51f36954e20,Tank Bund Road,IN,Bangalore,India,NaN,79,"[Tank Bund Road, Bangalore 560023, Karnātaka, ...","[{'label': 'display', 'lat': 12.97934222560161...",12.979342,77.590599,560023,Karnātaka,Fun Travel Deals,v-1592214545
2,"[{'id': '4bf58dd8d48988d17f941735', 'name': 'M...",False,51d9989a498e4376750d1c8d,NaN,IN,NaN,India,NaN,1031,[India],"[{'label': 'display', 'lat': 12.98773451082354...",12.987735,77.594791,NaN,NaN,Fun cinemas,v-1592214545
3,"[{'id': '4bf58dd8d48988d125941735', 'name': 'T...",False,4fed7618e4b0eb75846e9577,NaN,IN,NaN,India,NaN,1195,[India],"[{'label': 'display', 'lat': 12.98974220394271...",12.989742,77.592944,NaN,NaN,Vangal - Worldwide HQ of FUN,v-1592214545
4,"[{'id': '4bf58dd8d48988d18d941735', 'name': 'G...",False,4ebf6c257ee54e4cd288bf24,Phoenix MarketCity,IN,Bangalore,India,Whitefield Road,11575,"[Phoenix MarketCity (Whitefield Road), Bangalo...","[{'label': 'display', 'lat': 12.99675343923207...",12.996753,77.696471,NaN,Karnātaka,Fun Zone at Phoenix MarketCity,v-1592214545


In [106]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Fun Cinemas,Multiplex,Sigma Mall,IN,Bangalore,India,Cunningham Rd.,1091,"[Sigma Mall (Cunningham Rd.), Bangalore, Karnā...","[{'label': 'display', 'lat': 12.98832574437773...",12.988326,77.594769,NaN,Karnātaka,4b755b26f964a52005082ee3
1,Fun Travel Deals,Travel Lounge,Tank Bund Road,IN,Bangalore,India,NaN,79,"[Tank Bund Road, Bangalore 560023, Karnātaka, ...","[{'label': 'display', 'lat': 12.97934222560161...",12.979342,77.590599,560023,Karnātaka,515d73cff136b51f36954e20
2,Fun cinemas,Movie Theater,NaN,IN,NaN,India,NaN,1031,[India],"[{'label': 'display', 'lat': 12.98773451082354...",12.987735,77.594791,NaN,NaN,51d9989a498e4376750d1c8d
3,Vangal - Worldwide HQ of FUN,Tech Startup,NaN,IN,NaN,India,NaN,1195,[India],"[{'label': 'display', 'lat': 12.98974220394271...",12.989742,77.592944,NaN,NaN,4fed7618e4b0eb75846e9577
4,Fun Zone at Phoenix MarketCity,Gaming Cafe,Phoenix MarketCity,IN,Bangalore,India,Whitefield Road,11575,"[Phoenix MarketCity (Whitefield Road), Bangalo...","[{'label': 'display', 'lat': 12.99675343923207...",12.996753,77.696471,NaN,Karnātaka,4ebf6c257ee54e4cd288bf24
5,"Fun World, Near TV Tower, Bangalore",Recreation Center,Near TV Tower,IN,Bangalore,India,NaN,3239,"[Near TV Tower, Bangalore, Karnātaka, India]","[{'label': 'display', 'lat': 13.00820827484130...",13.008208,77.592293,NaN,Karnātaka,5439823d498e006131998d2a
6,FUN WORLD,Water Park,NaN,IN,NaN,India,NaN,3308,[India],"[{'label': 'display', 'lat': 13.00881283618616...",13.008813,77.592648,NaN,NaN,50d5b17d498e6fa62fdcb7c9
7,Fun World,Recreation Center,Near TV Tower,IN,Bangalore,India,NaN,3239,"[Near TV Tower, Bangalore, Karnātaka, India]","[{'label': 'display', 'lat': 13.00820827484130...",13.008208,77.592293,NaN,Karnātaka,543982bf498ed3511461f893
8,Fun Factory,General Entertainment,NaN,IN,Bangalore,India,NaN,3576,"[Bangalore 560026, Karnātaka, India]","[{'label': 'display', 'lat': 12.965316, 'lng':...",12.965316,77.561530,560026,Karnātaka,592ac6d851950e1087c3eeb0
9,"Star city, fun world",Bowling Alley,Palace grounds,IN,Bangalore,India,NaN,3080,"[Palace grounds, Bangalore, Karnātaka, India]","[{'label': 'display', 'lat': 13.00677094376065...",13.006771,77.592379,NaN,Karnātaka,4c566c14b7a31b8d570b4eda


In [108]:
bangalore_fun=dataframe_filtered['categories'].value_counts()
bangalore_fun.to_frame().rename(columns={'categories':"Count"},inplace=True)
bangalore_fun.index.name='categories'
bangalore_fun.head(15)

categories
Bank                       7
Office                     6
Theme Park                 3
Recreation Center          3
Gaming Cafe                2
Arcade                     2
Bowling Alley              2
General Entertainment      2
Movie Theater              2
Indoor Play Area           2
Indian Restaurant          2
Comfort Food Restaurant    1
Gym / Fitness Center       1
Travel Lounge              1
Surf Spot                  1
Name: categories, dtype: int64

In [109]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Bangalore

# add the fun venues as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

# 5) Results and Discussion <a name="results"></a> 

Given our cluster information for both Bangalore and Delhi, we see that Bangalore and its neighbourhoods are a great place for a foodie with large number of restaurants. Delhi neighborhoods and good for those who like Arts and Crafts, Museums, Water Parks and Pizza places. There is very less in terms of foreign cuisine restaurants in Delhi. Bangalore, on the other hand, has many diffent types of fun venues like Theme park, Recreation Centre, Gaming Café, Bowling Alley while Delhi has less of them. Delhi is inland and its neighbourhoods have proximity to Water Parks, Museums and Arts, and Crafts stores. Also we find that Delhi has many railway platforms unlike Bangalore which may prove to be a source of noise pollution and crowded place to its neighbourhoods.

# 6) Conclusion <a name="conclusion"></a>

In this projects we compared city of Delhi and Bangalore in India. It compares venues and location data of Delhi and Bangalore. With this we try to find how similar or dissimilar neighbourhoods of these cities are. We consider factors like types of venues like food, venues to visit. This project analyses the neighbourhoods in each of these two cities and tries to understand what is popular in them and what they have to offer to someone who is contemplating to make a choice on seeking a life in either of the metro cities.  

After analysis we find that Delhi has many museums, parks and is a place historic significance. It has good connectivity of railways. Bangalore does not have many historic significance venues. Bangalore has many different categories of venues to offers in comparison to Delhi. Both cities are great for food lovers.
